In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
from pathlib import Path
from torch.utils.data import Dataset
from PIL import Image

class FashionMNIST:
    def load_images(self, fpath):
        file = open(fpath, 'rb')
        
        file.read(4) # magic number
        nimages = int.from_bytes(file.read(4), 'big')
        rows = int.from_bytes(file.read(4), 'big')
        cols = int.from_bytes(file.read(4),'big')
        
        return np.array([Image.frombytes('L', (rows, cols), file.read(rows * cols))
                         for n in range(nimages)], dtype=object)
    
    def load_labels(self, fpath):
        file = open(fpath, 'rb')
        
        file.read(4) # magic number
        nlabels = int.from_bytes(file.read(4),'big')
        labels = file.read(nlabels)
        
        return np.frombuffer(labels, dtype=np.uint8).astype(np.int64)
    
    def __init__(self, images, labels=None):
        self.images = self.load_images(images)
        if labels is not None:
            self.labels = self.load_labels(labels)

root = Path('data', 'FashionMNIST', 'raw')
test_images = Path(root, 't10k-images-idx3-ubyte')
test_labels = Path(root, 't10k-labels-idx1-ubyte')
test_mnist = FashionMNIST(test_images, test_labels)

<ipython-input-2-0a379713a3cb>:14: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  return np.array([Image.frombytes('L', (rows, cols), file.read(rows * cols))


In [3]:
perfect = pd.DataFrame(data=[{'Id': ix, 'Class': pred}
                             for ix, pred in enumerate(test_mnist.labels)])
perfect.to_csv(Path('preds', 'perfect.csv'),
               index=False)

In [8]:
uid = '65ab42b0-158d-4ced-9a88-54087c519635'
checked = pd.read_csv('~/xdg/dl/{}.csv'.format(uid))
assert(len(perfect['Class']) == len(checked['Class']))

total = len(perfect['Class'])
correct = np.sum(perfect['Class'] == checked['Class'])
print('Accuracy: {} ({}/{})'.format(correct/total, correct, total))

Accuracy: 0.9404 (9404/10000)
